In [32]:
import time
import re
import random
import requests
from lxml import etree
import json


HEROES_URL = "https://game.gtimg.cn/images/lol/act/img/js/heroList/hero_list.js"
HERO_URL_T = "https://game.gtimg.cn/images/lol/act/img/js/hero/{}.js"
# PAGE_SIZE = 25
# TOTAL_PAGES = 10
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/143.0.0.0 Safari/537.36"
    )
}
role_dict = {
    'assassin': '刺客',
    'tank': '坦克',
    'support': '辅助',
    'fighter': '战士',
    'marksman': '射手',
    'mage': '法师'
}


def fetch_page(session: requests.Session, url) -> str:
    """
    Send an HTTP GET request to Douban Top250 page.

    :param session: requests.Session object for connection reuse
    :param start: pagination offset (0, 25, 50, ...)
    :return: HTML text of the page
    """ 
    response = session.get(url=url, headers=HEADERS, timeout=10)
    response.raise_for_status()
    return response.text

def main():
    with open('lolo_heros.txt', mode='w', encoding='utf-8') as f:
        session = requests.Session()
        html = fetch_page(session, HEROES_URL)
        json_obj = json.loads(html)
        heroes = json_obj['hero']
        for hero in heroes:
            hero_id = hero['heroId']
            hero_name = hero['name']
            hero_title = hero['title']
            hero_roles = hero['roles']
            # for e_role in hero_roles:
            #     cn_hero_roles.append(role_dict[e_role])
            # hero_cn_role = ','.join(cn_hero_roles)
            
            hero_url = HERO_URL_T.format(hero_id)
            html = fetch_page(session, hero_url)
            hero_obj = json.loads(html)
            spells = hero_obj['spells']
            spell_dict = {}
            for spell in spells:
                spell_name = spell['name']
                spell_key = spell['spellKey']
                spell_desc = spell['description'].replace('\n', '').strip()
                spell_dict[spell_key] = spell_name + ':' + spell_desc
                hero_info_line = '|'.join([hero_id, hero_name, hero_title, spell_key, spell_name, spell_desc])
                f.write(hero_info_line+'\n')
            # break

if __name__ == "__main__":
    main()